In [1]:
import supereeg as se
import numpy as np
import os
import glob as glob
import copy
from scipy.io import loadmat as load
from scipy.spatial.distance import pdist, cdist, squareform
from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
from visbrain.io import download_file, read_stc


In [2]:
def largest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, -n)[-n:]
    indices = indices[np.argsort(-flat[indices])]
    return np.unravel_index(indices, ary.shape)

In [3]:
def smallest_indices(ary, n):
    """Returns the n largest indices from a numpy array."""
    flat = ary.flatten()
    indices = np.argpartition(flat, n)[:n]
    indices = indices[np.argsort(flat[indices])]
    return np.unravel_index(indices, ary.shape)

In [4]:
def rbf(centers, widths, locs):
    """
    Radial basis function
    Parameters
    ----------
    centers : ndarray
        rbf coordinates (one row per RBF)
    widths : ndarray
        RBF radii
    locs : ndarray
        locations to evaluate the RBFs (one row per location)
        
    Returns
    ----------
    results : ndarray
        Matrix of RBF weights for each RBF (row), at each location (column)
    """    
    weights = np.exp(np.divide(-cdist(locs, centers, metric='euclidean') ** 2, np.tile(np.array(widths, ndmin=2), [locs.shape[0], 1])))
    return weights.T

In [5]:
#results_dir = os.path.join(os.getenv('HOME'), 'Desktop', 'timecorr_env', 'timecorr_paper', 'pieman', 'results')
results_dir = os.path.join('../../data/', 'results')

In [6]:
data_dir = os.path.join(results_dir, 'mean_corrs')
corrs_dir = os.path.join(data_dir, 'corrs')

In [7]:
neurosynth_dir ='../figs/neurosynth_data/'

In [10]:
bo_dir = os.path.join(neurosynth_dir, 'bos')
if not os.path.exists(bo_dir):
    os.mkdir(bo_dir)

In [9]:
posterior = load(os.path.join(results_dir, '../data/pieman_posterior_K700.mat'))
centers = posterior['posterior']['centers'][0][0][0][0][0]
widths = np.array(list(posterior['posterior']['widths'][0][0][0][0][0][:, 0].T))

In [14]:
template = se.helpers._gray(res=2)

In [38]:
levels = np.arange(0, 15, 1)
conditions = ['intact', 'paragraph', 'rest', 'word']

In [39]:
top_n = 10

In [33]:

for l in levels:
    for c in conditions:
        conds = glob.glob(os.path.join(data_dir, f'level_{l}', f'{c}.npy'))
        g_m = np.load(conds[0])

        networks = copy.copy(g_m)
        np.fill_diagonal(networks, 0)
        net_inds = largest_indices(np.triu(networks), top_n)
        pos_mask = networks[net_inds] > 0
        net_inds = np.concatenate((net_inds[0][pos_mask], net_inds[1][pos_mask]))
        temp_locs = centers[net_inds]
        temp_widths = widths[net_inds]
        
        w = rbf(temp_locs, temp_widths, template.get_locs().values)
        b = se.Brain(data=np.array(np.sum(w, axis=0), ndmin=2), locs=template.get_locs(), minimum_voxel_size=2)
        
        data = b.get_data().values.ravel()
        xyz = b.get_locs().values
        xyz = xyz[data>.999]
        data = data[data>.999]
        data = np.ones(data.shape)+l
        
        bo = se.Brain(data=data, locs=xyz)
        outfile = c+ '_' + str(l+1)
        bo.save(os.path.join(bo_dir, outfile + '_largest'))

In [32]:

for l in levels:
    for c in conditions:
        conds = glob.glob(os.path.join(data_dir, f'level_{l}', f'{c}.npy'))
        g_m = np.load(conds[0])

        networks = copy.copy(g_m)
        np.fill_diagonal(networks, 0)
        net_inds = smallest_indices(np.triu(networks), top_n)
        neg_mask = networks[net_inds] < 0
        net_inds = np.concatenate((net_inds[0][neg_mask], net_inds[1][neg_mask]))
        temp_locs = centers[net_inds]
        temp_widths = widths[net_inds]
        
        w = rbf(temp_locs, temp_widths, template.get_locs().values)
        b = se.Brain(data=np.array(np.sum(w, axis=0), ndmin=2), locs=template.get_locs(), minimum_voxel_size=2)
        
        data = b.get_data().values.ravel()
        xyz = b.get_locs().values
        xyz = xyz[data>.999]
        data = data[data>.999]
        data = np.ones(data.shape)+l
        
        bo = se.Brain(data=data, locs=xyz)
        outfile = c+ '_' + str(l+1)
        bo.save(os.path.join(bo_dir, outfile + '_smallest'))

In [40]:
for l in levels:
    for c in conditions:
        conds = glob.glob(os.path.join(data_dir, f'level_{l}', f'{c}.npy'))
        g_m = np.load(conds[0])

        networks = copy.copy(g_m)
        np.fill_diagonal(networks, 0)
        net_inds = largest_indices(np.triu(np.abs(networks)), top_n)
        net_inds = np.concatenate((net_inds[0], net_inds[1]))
        temp_locs = centers[net_inds]
        temp_widths = widths[net_inds]
        
        w = rbf(temp_locs, temp_widths, template.get_locs().values)
        b = se.Brain(data=np.array(np.sum(w, axis=0), ndmin=2), locs=template.get_locs(), minimum_voxel_size=2)
        
        data = b.get_data().values.ravel()
        xyz = b.get_locs().values
        xyz = xyz[data>.999]
        data = data[data>.999]
        data = np.ones(data.shape)+l
        
        bo = se.Brain(data=data, locs=xyz)
        outfile = c+ '_' + str(l+1)
        bo.save(os.path.join(bo_dir, outfile + '_largest_abs'))
        
        